<a href="https://colab.research.google.com/github/hws2002/Deep_Learning_with_Keras/blob/main/Chapter11/Chapter11_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -r aclImdb
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

rm: cannot remove 'aclImdb': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  5069k      0  0:00:16  0:00:16 --:--:-- 9352k


수동으로 구현한 train_dataset, val_dataset

In [2]:
import os

base_path = os.path.join("aclImdb")

base_dir = 'aclImdb'
train_dir = 'train/'
test_dir = 'test/'

train_dataset = []
val_dataset = [] # 20%만 떼어가자

train_pos_path = os.path.join(base_path, train_dir, 'pos')
train_neg_path = os.path.join(base_path, train_dir, 'neg')

train_pos_listdir = os.listdir(train_pos_path)
num_val_pos_data = int(0.2 * len(train_pos_listdir))
train_neg_listdir = os.listdir(train_neg_path)
num_val_neg_data = int(0.2 * len(train_neg_listdir))

for filename in train_pos_listdir[:-num_val_pos_data]:
  with open(os.path.join(train_pos_path,filename),'r') as f:
    train_dataset.append(f.read())
for filename in train_pos_listdir[-num_val_pos_data:]:
  with open(os.path.join(train_pos_path,filename),'r') as f:
    val_dataset.append(f.read())

num_train_pos_data = len(train_dataset)

for filename in train_neg_listdir[:-num_val_neg_data]:
  with open(os.path.join(train_neg_path,filename),'r') as f:
    train_dataset.append(f.read())

for filename in train_neg_listdir[-num_val_neg_data:]:
  with open(os.path.join(train_neg_path,filename),'r') as f:
    val_dataset.append(f.read())
num_train_neg_data = len(train_dataset) - num_train_pos_data

print(len(train_dataset))
print(len(val_dataset))

20000
5000


In [3]:
import numpy as np
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset)
val_dataset = tf.data.Dataset.from_tensor_slices(val_dataset)

for i , element in enumerate(train_dataset):
  print(element)
  if i>= 3:
    break


for i , element in enumerate(val_dataset):
  print(element)
  if i>= 3:
    break

tf.Tensor(b'Not the most successful television project John Cleese ever did, "Strange Case" has the feel of a first draft that was rushed into production before any revisions could be made. There are some silly ideas throughout and even a few clever ones, but the story as a whole unfortunately doesn\'t add up to much.<br /><br />Arthur Lowe is a hoot, though, as Dr. Watson, bionic bits and all. "Good Lord."', shape=(), dtype=string)
tf.Tensor(b"George Cukor directs a brooding and cynical classic. The distinctive Ronald Coleman is at his best in this piece of Noir about an actor who loses himself in his roles. The acclaimed Anthony John(Colman)has driven his wife Brita(Signe Hasso)away with his highly fueled temper and erratic behavior. But the two manage to continue working together to please their audiences. Things begin to change as John is becoming bored with his career; he reluctantly agrees to play Othello. He gets deep into character as a jealous and murderous man. He begins walk

텍스트 파일에 대해 text_dataset_from_directory 유틸리티로 쉽고 빠르게 text dataset을 만들 수 있다.

먼저 훈련 텍스트 파일에서 20%를 새로운 디렉터리 aclImdb/val로 덜어 내고

In [4]:
import pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / 'val'
train_dir = base_dir / 'train'

for category in ("neg","pos"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_val_samples = int( 0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname ,
                val_dir / category / fname)

text_dataset_from_directory 유틸리티로 훈련, 검증, 테스트를 위한 3개의 Dataset 객체를 만들 수 있다.

In [5]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size = batch_size
)

val_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/val', batch_size = batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/test', batch_size = batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
batched_train_ds = train_ds.batch(32)

for i, element in enumerate(train_ds):
  print(element)
  if i >= 2:
    break

(<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'this movie is a very relaxed, romantic-comedy, which is thoroughly enjoyable. Meg Ryan does a very good job as the genius niece of Albert Einstein, though she does believe in her own skills. Tim Robbins does an equally good job as the mechanic who falls in love with her when she comes into his shop with her fianc\xc3\xa9 after her car stuffs up. I loved Walter Matthau as the one and only Albert Einstein. This movie just has a very relaxing feel to it, while still keeping some sort of seriousness to it (if that is actually possible, it happens here).<br /><br />I personally found this movie extremely entertaining, especially the old scientists - i thought they were fab and hilarious! This movie seems to have been underestimated beyond comprehension. If you have a cheeky sense of humour, this is the movie for you!',
       b"Check out the film's website, more time was put into making that than in the writing of the script for this m

# 단어를 집합으로 처리하기 : BoW 방식
순서를 무시하고 토큰의 집합으로 다루어 보자.  
개별 단어(유니그램) 을 사용하거나 또는 연속된 토큰 그룹(N그램) 으로 국부적인  순서 정보를 유지할 수 있음.  


TextVectorization 층으로 처리하여 멀티-핫-인코딩된 이진 단어 벡터를 만들어 보자 (유니그램)

In [ ]:
from keras.layers import TextVectorization

text_vectorization = TextVectorization(
    max_tokens = 20000, # 가장 많이 등장하는 2만개 단어로 어휘 사전을 제한
    output_mode = "multi_hot", # 멀티-핫 이진 벡터로 출력 토큰을 인코딩
)

text_only_train_ds = train_ds.map(lambda x, y : x)
text_vectorization.adapt(text_only_train_ds)

binary_1gram_train_ds = train_ds.map(
    lambda x,y : (text_vectorization(x), y),
    num_parallel_calls = 4
)

binary_1gram_val_ds = val_ds.map(
    lambda x,y : (text_vectorization(x), y),
    num_parallel_calls = 4
)

binary_1gram_test_ds = test_ds.map(
    lambda x,y : (text_vectorization(x), y),
    num_parallel_calls = 4
)


for i,element in enumerate(binary_1gram_train_ds):
  print(element)
  if i >= 2:
    break

(<tf.Tensor: shape=(32, 20000), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 1], dtype=int32)>)
(<tf.Tensor: shape=(32, 20000), dtype=int64, numpy=
array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 0, 1], dtype=int32)>)
(<tf.Tensor: shape=(32, 20000), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0,

이제 이 절의 모든 예제에서 사용할 모델 생성 함수를 만들어 보자

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens = 20000, hidden_dims = 16):
  inputs = keras.Input(shape = (max_tokens,))
  x = layers.Dense(hidden_dims, activation = "relu")(inputs)
  x = layers.Dropout(0.5)(x)
  outputs = layers.Dense(1, activation = "sigmoid")(x)
  model = keras.Model(inputs, outputs)

  model.compile(
      optimizer = "rmsprop",
      loss = 'binary_crossentropy',
      metrics = ['accuracy']
  )

  return model

마지막으로 훈련을 진행해보자

In [ ]:
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras",
                                    save_best_only = True)
]

history = model.fit(
    binary_1gram_train_ds.cache(),
    epochs = 10,
    validation_data = binary_1gram_val_ds.cache(), # 메모리에 캐싱
    callbacks = callbacks
)

In [ ]:
model = keras.models.load_model("binary_1gram.keras")
print(f"테스트 정확도 : {model.evaluate(binary_1gram_test_ds)[1]}:.3f")

782/782 ━━━━━━━━━━━━━━━━━━━━ 644s 822ms/step - accuracy: 0.8846 - loss: 0.2924
테스트 정확도 : 0.8881999850273132:.3f


## 이진 인코딩을 사용한 바이그램
단일 단어가 아닌 N-gram을 사용하여 국부적인 순서정보를 BoW 표현에 추가해보자    
TextVectorization 층은 바이그램, 트라이그램을 포함하여 임의의 N-gram을 반환할 수 있음.  


In [ ]:
text_vectorization = TextVectorization(
    ngrams = 2,
    max_tokens = 20000,
    output_mode = 'multi_hot',
)

이진 인코딩된 바이그램에서 훈련한 모델의 성능을 확인해 보자

In [ ]:
text_vectorization.adapt(text_only_train_ds)

In [ ]:
binary_2gram_train_ds = train_ds.map(
    lambda x ,y : (text_vectorization(x), y),
    num_parallel_calls = 4
)

binary_2gram_val_ds = val_ds.map(
    lambda x ,y : (text_vectorization(x), y),
    num_parallel_calls = 4
)

binary_2gram_test_ds = test_ds.map(
    lambda x ,y : (text_vectorization(x), y),
    num_parallel_calls = 4
)

In [ ]:
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_2gram.keras",
                                    save_best_only = True)
]

model.fit(
    binary_2gram_train_ds.cache(),
    validation_data = binary_2gram_val_ds.cache(),
    epochs = 10,
    callbacks = callbacks
)

model = keras.models.load_model("binary_2gram.keras")
print(f"테스트 정확도 : {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 20000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │         320,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1237s 2s/step - accuracy: 0.7701 - loss: 0.4850 - val_accuracy: 0.8932 - val_loss: 0.2733
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 51s 8ms/step - accuracy: 0.9088 - loss: 0.2533 - val_accuracy: 0.8970 - val_loss: 0.2750
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9288 - loss: 0.2234 - val_accuracy: 0.9012 - val_loss: 0.2933
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9416 - loss: 0.1924 - val_accuracy: 0.8992 - val_loss: 0.2969
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9465 - loss: 0.1779 - val_accuracy: 0.8956 - val_loss: 0.3215
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.9495 - loss: 0.1705 - val_accuracy: 0.8998 - val_loss: 0.3332
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9528 - loss: 0.1780 - val_accuracy: 0.8922 - val_loss: 0.3490
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9535 - loss: 0.1704 - val_accuracy

# TF_IDF 인코딩을 사용한 바이그램
이 표현에서는 개별 단어나 N-그램의 등장 횟수를 카운트한 정보를 추가할 수 있음.  
즉, 텍스트에 대한 단어의 히스토그램(histogram)을 사용함.  


In [ ]:
text_vectorization = TextVectorization(
    ngrams = 2,
    max_tokens = 20000,
    output_mode = "count"
)

In [ ]:
text_vectorization = TextVectorization(
    ngrams = 2,
    max_tokens = 20000,
    output_mode = "tf_idf",
)

이 방식으로 새 모델을 훈련해 보자

In [ ]:
text_vectorization.adapt(text_only_train_ds)

tfidf_2gram_train_ds = train_ds.map(
    lambda x ,y : (text_vectorization(x), y),
    num_parallel_calls = 4
)

tfidf_2gram_val_ds = val_ds.map(
    lambda x ,y : (text_vectorization(x), y),
    num_parallel_calls = 4
)

tfidf_2gram_test_ds = test_ds.map(
    lambda x ,y : (text_vectorization(x), y),
    num_parallel_calls = 4
)

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.keras",
                                    save_best_only = True)
]

model.fit(
    tfidf_2gram_train_ds.cache(),
    validation_data = tfidf_2gram_val_ds.cache(),
    epochs = 10,
    callbacks = callbacks
)

model = keras.models.load_model("tfidf_2gram.keras")
print(f"테스트 정확도 : {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 20000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 16)                  │         320,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1194s 2s/step - accuracy: 0.7029 - loss: 0.6119 - val_accuracy: 0.8780 - val_loss: 0.3127
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 33s 6ms/step - accuracy: 0.8500 - loss: 0.3630 - val_accuracy: 0.8784 - val_loss: 0.3043
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8768 - loss: 0.3065 - val_accuracy: 0.8734 - val_loss: 0.3307
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.8881 - loss: 0.2797 - val_accuracy: 0.8838 - val_loss: 0.3213
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9024 - loss: 0.2583 - val_accuracy: 0.8670 - val_loss: 0.3390
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9016 - loss: 0.2397 - val_accuracy: 0.8520 - val_loss: 0.3767
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.9050 - loss: 0.2356 - val_accuracy: 0.8728 - val_loss: 0.3569
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9095 - loss: 0.2312 - val_accuracy:

# 단어를 시퀀스로 처리하기 : 시퀀스 모델 방식

시퀀스 모델을 구현하려면 먼저 입력 샘플을 정수 인덱스의 시퀀스로 표현해야 함.(하나의 정수가 하나의 단어를 나타냄) 그다음 각 정수를 벡터로 매핑하여 벡터 시퀀스를 얻고, 마지막으로 이 벡터 시퀀스를 1D 컨브넷, RNN, 트랜스포머와 같이 인접한 벡터의 특징을 비교할 수 있는 층에 전달함.

## 정수 시퀀스를 반환하는 데이터셋

먼저 정수 시퀀스를 반환하는 데이터셋을 준비하자

In [17]:
from keras import layers
text_only_train_ds = train_ds.map(lambda x, y : x)

In [18]:
max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens = max_tokens,
    output_mode = "int",
    output_sequence_length = max_length, # 적당한 입력 크기를 유지하기 위해 입력에서 600개 단어 이후는 잘라 버림
    # 평균 리뷰 길이가 233개의 단어고 600개의 단어보다 긴 리뷰는 5%뿐이므로 합리적인 선택임.
)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x,y : (text_vectorization(x), y),
    num_parallel_calls = 4)

int_val_ds = val_ds.map(
    lambda x,y : (text_vectorization(x), y),
    num_parallel_calls = 4)

int_test_ds = test_ds.map(
    lambda x,y : (text_vectorization(x), y),
    num_parallel_calls = 4)

In [ ]:
for i, element in enumerate(int_train_ds):
  print(element)
  if i>=2 :
    break

(<tf.Tensor: shape=(32, 600), dtype=int64, numpy=
array([[   2,   87, 1227, ...,    0,    0,    0],
       [ 234,   11,  427, ...,    0,    0,    0],
       [  11,  387,    7, ...,    0,    0,    0],
       ...,
       [  10,  118,   57, ...,    0,    0,    0],
       [ 256,    9,   44, ...,    0,    0,    0],
       [   2,  152,   43, ...,    0,    0,    0]])>, <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=int32)>)
(<tf.Tensor: shape=(32, 600), dtype=int64, numpy=
array([[ 11,   7,   2, ...,   0,   0,   0],
       [  8,  34, 565, ...,   0,   0,   0],
       [  2,  18,  14, ...,   0,   0,   0],
       ...,
       [169, 715,  60, ...,   0,   0,   0],
       [ 10, 207,  11, ...,   0,   0,   0],
       [ 10,  95,  22, ...,   0,   0,   0]])>, <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 0

## 원-핫 인코딩

이제 모델을 만들어 보자.
정수 시퀀스를 벡터 시퀀스로 바꾸는 가장 간단한 방법은 정수를 원-핫 인코딩하는 것임(각 차원은 어휘 사전에 있는 하나의 단어를 표현)

In [ ]:
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Lambda(
    lambda x : tf.one_hot(x, depth = max_tokens),
    output_shape = lambda s : (s[0], s[1], max_tokens) )(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only = True)
]

model.fit(
    int_train_ds.cache(),
    validation_data = int_val_ds.cache(),
    epochs = 10,
    callbacks = callbacks
)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda (Lambda)                      │ (None, None, 20000)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 64)                  │       5,128,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,128,513 (19.56 MB)

 Trainable params: 5,128,513 (19.56 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 233s 367ms/step - accuracy: 0.5734 - loss: 0.6624 - val_accuracy: 0.8272 - val_loss: 0.4725
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 261s 369ms/step - accuracy: 0.8350 - loss: 0.4249 - val_accuracy: 0.8630 - val_loss: 0.3548
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 228s 366ms/step - accuracy: 0.8826 - loss: 0.3286 - val_accuracy: 0.8862 - val_loss: 0.2936
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 228s 364ms/step - accuracy: 0.9040 - loss: 0.2766 - val_accuracy: 0.8852 - val_loss: 0.3071
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 262s 365ms/step - accuracy: 0.9185 - loss: 0.2382 - val_accuracy: 0.8870 - val_loss: 0.3175
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 229s 366ms/step - accuracy: 0.9307 - loss: 0.2104 - val_accuracy: 0.8594 - val_loss: 0.3349
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 227s 364ms/step - accuracy: 0.9362 - loss: 0.2005 - val_accuracy: 0.8838 - val_loss: 0.3812
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 263s 365ms/step - accuracy: 0.9484 -

이 모델의 훈련은 입력크기가 매우 크기때문에 (샘플당 600개의 단어가 있고 각 단어는 2만 개의 단어 중 하나), 훈련이 매우 느림

또한, 테스트 정확도가 좋지 않음

In [ ]:
model = keras.models.load_model("one_hot_bidir_lstm.keras", safe_mode = False)
print(f"테스트 정확도 : {model.evaluate(int_test_ds)[1]:.3f}")

NameError: Exception encountered when calling Lambda.call().

[1mname 'max_tokens' is not defined[0m

Arguments received by Lambda.call():
  • args=('<KerasTensor shape=(None, None), dtype=int64, sparse=False, name=input_layer>',)
  • kwargs={'mask': 'None'}

## 단어 임베딩

### 단어 임베딩을 만드는 첫번째 방법 - Embedding 층으로 단어 임베딩 학습하기

In [ ]:
embedding_layer = layers.Embedding(
    input_dim = max_tokens,
    output_dim = 256
)

In [ ]:
inputs = keras.Input(shape = (None,), dtype = 'int64')
embedded = layers.Embedding(
    input_dim = max_tokens,
    output_dim = 256
)(inputs)

x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()


callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_lstm.keras",
                                    save_best_only = True)
]

model.fit(
    int_train_ds.cache(),
    validation_data = int_val_ds.cache(),
    epochs = 10,
    callbacks = callbacks
)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, None, 256)           │       5,120,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 64)                  │          73,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,194,049 (19.81 MB)

 Trainable params: 5,194,049 (19.81 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 30s 43ms/step - accuracy: 0.6082 - loss: 0.6389 - val_accuracy: 0.8286 - val_loss: 0.4178
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 40s 43ms/step - accuracy: 0.8228 - loss: 0.4400 - val_accuracy: 0.8044 - val_loss: 0.4198
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - accuracy: 0.8681 - loss: 0.3456 - val_accuracy: 0.8674 - val_loss: 0.3336
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 39s 39ms/step - accuracy: 0.8895 - loss: 0.3017 - val_accuracy: 0.8662 - val_loss: 0.3591
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 39ms/step - accuracy: 0.9155 - loss: 0.2505 - val_accuracy: 0.8754 - val_loss: 0.3585
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 25s 41ms/step - accuracy: 0.9241 - loss: 0.2212 - val_accuracy: 0.8706 - val_loss: 0.3618
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 39ms/step - accuracy: 0.9394 - loss: 0.1962 - val_accuracy: 0.8772 - val_loss: 0.3645
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 41s 40ms/step - accuracy: 0.9488 - loss: 0.1607 - 

In [ ]:
model = keras.models.load_model("embeddings_bidir_lstm.keras")
print(f"테스트 정확도  : {model.evaluate(int_test_ds)[1]:.3f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.8456 - loss: 0.3676
테스트 정확도  : 0.847


이제 LSTM이 20000차원이 아니라 256차원 벡터를 처리하기 때문에 이 모델은 원-핫 모델보다 훨씬 빠르고 테스트 정확도는 비슷함  
하지만 여전히 기본적인 바이그램 모델의 결과보다 차이가 남. 이유 중 하나는 이 모델이 약간 적은 데이터를 사용하기 때문임. 바이그램 모델은 전체 리뷰를 처리하지만, 이 시퀀스 모델은 600개의 단어 이후 시퀀스는 잘라 버리기 때문

### 패딩과 마스킹

In [ ]:
embedding_layer = layers.Embedding(
    input_dim = 10,
    output_dim = 256,
    mask_zero = True
)

some_input = [
    [4, 3, 2, 1, 0, 0, 0],
    [5, 4, 3, 2, 1, 0, 0],
    [2, 1, 0, 0, 0, 0, 0]
]

mask = embedding_layer.compute_mask(some_input)

In [ ]:
inputs = keras.Input(shape = (None,), dtype = 'int64')
embedded = layers.Embedding(
    input_dim = max_tokens, output_dim = 256, mask_zero = True)(inputs)

x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()


callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_lstm_with_masking.keras",
                                    save_best_only = True)
]

model.fit(
    int_train_ds.cache(),
    validation_data = int_val_ds.cache(),
    epochs = 10,
    callbacks = callbacks
)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_5 (Embedding)   │ (None, None, 256)      │      5,120,000 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, None)           │              0 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_2           │ (None, 64)             │         73,984 │ embedding_5[0][0],     │
│ (Bidirectional)           │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 64)             │              0 │ bidirectional_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 1)              │             65 │ dropout_2[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 5,194,049 (19.81 MB)

 Trainable params: 5,194,049 (19.81 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 32s 47ms/step - accuracy: 0.6746 - loss: 0.5814 - val_accuracy: 0.8622 - val_loss: 0.3367
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 43s 51ms/step - accuracy: 0.8618 - loss: 0.3309 - val_accuracy: 0.8774 - val_loss: 0.2936
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 35s 41ms/step - accuracy: 0.9001 - loss: 0.2541 - val_accuracy: 0.8494 - val_loss: 0.4023
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 38ms/step - accuracy: 0.9226 - loss: 0.2061 - val_accuracy: 0.8278 - val_loss: 0.4979
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - accuracy: 0.9409 - loss: 0.1586 - val_accuracy: 0.8690 - val_loss: 0.3856
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - accuracy: 0.9605 - loss: 0.1172 - val_accuracy: 0.8754 - val_loss: 0.4246
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 38ms/step - accuracy: 0.9668 - loss: 0.0907 - val_accuracy: 0.8722 - val_loss: 0.3853
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 38ms/step - accuracy: 0.9781 - loss: 0.0661 - 

## 단어 임베딩을 만드는 두번째 방법 - 사전 훈련된 단어 임베딩 사용하기

GloVec 임베딩을 케라스 모델에서 어떻게 사용하는지 알아보자

먼저 GloVec 파일을 내려받고 파싱해야 함.  
그 후 단어 벡터를 케라스 Embedding 층으로 로드하여 새로운 모델을 만들어 보자

In [8]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2025-01-23 12:44:54--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-01-23 12:44:55--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-01-23 12:44:55--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [11]:
import numpy as np

path_to_glove_file = 'glove.6B.100d.txt' # embedding_dim이 100이다.
embeddings_index = {}
with open(path_to_glove_file) as f:
  for line in f:
    word, coef = line.split(maxsplit = 1)
    coefs = np.fromstring(coef, "f", sep = " ")
    embeddings_index[word] = coefs

print(f"단어 벡터 개수 : {len(embeddings_index)}")

단어 벡터 개수 : 400000


In [15]:
for i, (word, wordvec) in enumerate(embeddings_index.items()):
  print(word, wordvec)
  if i >= 2:
    break

the [-0.038194 -0.24487   0.72812  -0.39961   0.083172  0.043953 -0.39141
  0.3344   -0.57545   0.087459  0.28787  -0.06731   0.30906  -0.26384
 -0.13231  -0.20757   0.33395  -0.33848  -0.31743  -0.48336   0.1464
 -0.37304   0.34577   0.052041  0.44946  -0.46971   0.02628  -0.54155
 -0.15518  -0.14107  -0.039722  0.28277   0.14393   0.23464  -0.31021
  0.086173  0.20397   0.52624   0.17164  -0.082378 -0.71787  -0.41531
  0.20335  -0.12763   0.41367   0.55187   0.57908  -0.33477  -0.36559
 -0.54857  -0.062892  0.26584   0.30205   0.99775  -0.80481  -3.0243
  0.01254  -0.36942   2.2167    0.72201  -0.24978   0.92136   0.034514
  0.46745   1.1079   -0.19358  -0.074575  0.23353  -0.052062 -0.22044
  0.057162 -0.15806  -0.30798  -0.41625   0.37972   0.15006  -0.53212
 -0.2055   -1.2526    0.071624  0.70565   0.49744  -0.42063   0.26148
 -1.538    -0.30223  -0.073438 -0.28312   0.37104  -0.25217   0.016215
 -0.017099 -0.38984   0.87424  -0.72569  -0.51058  -0.52028  -0.1459
  0.8278    0.270

In [19]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary() # TextVectorization층에 인덱싱된 단어를 추출(즉, 우리의 testdataset에 있던)
word_index = dict(zip(vocabulary, range(len(vocabulary))))
word_index

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
  if i < max_tokens:
    embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

이제 Constant 초기화를 사용하여 Embedding 층에 사전 훈련된 임베딩을 로드함.  
훈련하는 동안 사전 훈련된 표현이 변경되지 않도록 trainable=False로 이 층을 동결함.

In [20]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer = keras.initializers.Constant(embedding_matrix),
    trainable = False,
    mask_zero = True,
)

이제 100차원의 사전 훈련된 GloVe 임베딩을 사용해 보자

In [21]:
inputs = keras.Input(shape = (None,), dtype = 'int64')
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()


callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_bidir_sequence_model.keras",
                                    save_best_only = True)
]

model.fit(
    int_train_ds.cache(),
    validation_data = int_val_ds.cache(),
    epochs = 10,
    callbacks = callbacks
)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, None, 100)      │      2,000,000 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, None)           │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 64)             │         34,048 │ embedding[0][0],       │
│ (Bidirectional)           │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 64)             │              0 │ bidirectional[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 1)              │             65 │ dropout[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,034,113 (7.76 MB)

 Trainable params: 34,113 (133.25 KB)

 Non-trainable params: 2,000,000 (7.63 MB)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 38s 54ms/step - accuracy: 0.6201 - loss: 0.6388 - val_accuracy: 0.7630 - val_loss: 0.4890
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 32s 44ms/step - accuracy: 0.7805 - loss: 0.4728 - val_accuracy: 0.8094 - val_loss: 0.4133
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 44s 49ms/step - accuracy: 0.8154 - loss: 0.4178 - val_accuracy: 0.8294 - val_loss: 0.3816
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 44ms/step - accuracy: 0.8375 - loss: 0.3824 - val_accuracy: 0.8202 - val_loss: 0.3895
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 39s 41ms/step - accuracy: 0.8518 - loss: 0.3535 - val_accuracy: 0.8106 - val_loss: 0.4345
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 46s 49ms/step - accuracy: 0.8651 - loss: 0.3310 - val_accuracy: 0.8524 - val_loss: 0.3579
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 40s 47ms/step - accuracy: 0.8745 - loss: 0.3134 - val_accuracy: 0.8580 - val_loss: 0.3576
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 27s 44ms/step - accuracy: 0.8845 - loss: 0.2937 - 

In [22]:
model = keras.models.load_model("glove_embeddings_bidir_sequence_model.keras")
print(f"테스트 정확도  : {model.evaluate(int_test_ds)[1]:.3f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.8657 - loss: 0.3298
테스트 정확도  : 0.861


이 작업에서는 사전 훈련된 임베딩이 별로 도움이 되지는 않는 것 같음. 작업에 특화된 임베딩 공간을 밑바닥부터 학습하기에 충분한 샘플이 데이터셋에 있기 때문.  
하지만 작은 데이터셋을 다룰 때는 사전 훈련된 임베딩을 사용하면 크게 도움이 될 수 있음.
